# Global Stuffs

In [ ]:
# use env bl17_2 to run this script

import param
import panel as pn

import holoviews as hv
import imageio
import os, glob, time

import json
import pandas as pd

import tqdm
import numpy as np


import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import cm

from tkinter import font
import numpy as np
import matplotlib.pyplot as plt
import pyFAI

import numpy as np
import pyFAI
import os
import matplotlib.pyplot as plt

# Functions

In [ ]:
def set_plot_style(axs, fonts, xlabel, ylabel):
    axs.set_xlabel(xlabel, fontsize=fonts)
    axs.set_ylabel(ylabel, fontsize=fonts)
    axs.tick_params(axis='both', which='major', direction='out', length=4, width=1)
    axs.tick_params(which='minor', width=1, size=2)  # Adjust size as needed
    axs.minorticks_on() # Turn on minor ticks
    
    #axs.grid(False, which='both', axis='both', linestyle='--', linewidth=0.5)
    axs.set_facecolor('white')
    axs.spines['top'].set_linewidth(1)
    axs.spines['right'].set_linewidth(1)
    axs.spines['bottom'].set_linewidth(1)
    axs.spines['left'].set_linewidth(1)
    axs.tick_params(axis='x', labelsize=fonts)
    axs.tick_params(axis='y', labelsize=fonts)

    return axs

## save subtracted data

In [ ]:
import os
import pandas as pd

def save_subtracted_txt(q, I_sub, I_sub_sigma, I_raw, fname_sample, fname_bkg, fname_empty, timer, bstop, ctemp, I0, out_dir):
    """
    Save subtracted WAXS data to a .dat text file with headers.

    Parameters:
    - q: array-like, q values in nm^-1
    - I_sub: array-like, background-subtracted intensity
    - I_sub_sigma: array-like, sigma of subtracted intensity
    - I_raw: array-like, raw or normalized sample intensity
    - fname_sample: str, sample filename
    - fname_bkg: str, background filename
    - fname_empty: str, empty scan filename
    - timer: float or str, exposure time
    - bstop: float or str, beamstop position
    - ctemp: float or str, camera temperature
    - I0: float or str, incident beam intensity
    - out_dir: str, output directory
    """

    # Create DataFrame
    data = {
        "q_nm^-1": q,
        "I_avg_sub": I_sub,
        "I_avg_sub_sigma": I_sub_sigma,
        "I_sample": I_raw,
    }
    df = pd.DataFrame(data)

    # Output file name
    out_file = os.path.join(out_dir, f"{fname_sample.split('.dat')[0]}_sub.dat")

    # Header lines
    inst_info = f"Timer: {timer}, bstop: {bstop}, ctemp: {ctemp}, I0: {I0}"
    headers = [
        f"filename: {fname_sample}",
        f"background : {fname_bkg}",
        f"Empty : {fname_empty}",
        inst_info,
        "fit_data",
        "q_nm^-1 ------ I_avg_subtracted ------ I_avg_subtracted_sigma ------ I_Normalized",
    ]
    commented = ['# ' + line for line in headers]

    # Write file
    with open(out_file, 'w') as f:
        f.write('\n'.join(commented) + '\n')
        df.to_csv(f, sep='\t', index=False, header=False)


In [ ]:
import os
import pandas as pd

def save_subtracted_txt(q, I_sub, I_sub_sigma, I_raw, fname_sample, fname_bkg, fname_empty, timer, bstop, ctemp, I0, out_dir):
    """
    Save subtracted WAXS data to a .dat text file with headers.

    Parameters:
    - q: array-like, q values in nm^-1
    - I_sub: array-like, background-subtracted intensity
    - I_sub_sigma: array-like, sigma of subtracted intensity
    - I_raw: array-like, raw or normalized sample intensity
    - fname_sample: str, sample filename
    - fname_bkg: str, background filename
    - fname_empty: str, empty scan filename
    - timer: float or str, exposure time
    - bstop: float or str, beamstop position
    - ctemp: float or str, camera temperature
    - I0: float or str, incident beam intensity
    - out_dir: str, output directory
    """

    # Create DataFrame
    data = {
        "q_nm^-1": q,
        "I_avg_sub": I_sub,
        "I_avg_sub_sigma": I_sub_sigma,
        "I_sample": I_raw,
    }
    df = pd.DataFrame(data)

    # Output file name
    out_file = os.path.join(out_dir, f"{fname_sample.split('.dat')[0]}_sub.dat")

    # Header lines
    inst_info = f"Timer: {timer}, bstop: {bstop}, ctemp: {ctemp}, I0: {I0}"
    headers = [
        f"filename: {fname_sample}",
        f"background : {fname_bkg}",
        f"Empty : {fname_empty}",
        inst_info,
        "fit_data",
        "q_nm^-1 ------ I_avg_subtracted ------ I_avg_subtracted_sigma ------ I_Normalized",
    ]
    commented = ['# ' + line for line in headers]

    # Write file
    with open(out_file, 'w') as f:
        f.write('\n'.join(commented) + '\n')
        df.to_csv(f, sep='\t', index=False, header=False)


## read HDF5 file


In [ ]:
def read_h5_data(base_path, samp_folder, keyword, azimuthal=True):
    """
    Read azimuthal or radial data from HDF5 files generated by integrate_tif_files.

    Parameters:
    -----------
    base_path : str
        Root path to the data.
    samp_folder : str
        Folder name corresponding to the sample.
    keyword : str
        Keyword like "PS_500C" etc.
    azimuthal : bool, optional
        If True, reads azimuthal data; otherwise reads radial data.

    Returns:    
    --------
    data_list : list of dict
        Each entry corresponds to one dataset stored in the HDF5 file.
    """
    import h5py
    import os
    import glob

    # Construct the expected folder path
    target_folder = os.path.join(base_path, samp_folder)
    if not os.path.exists(target_folder):
        raise FileNotFoundError(f"Target folder does not exist: {target_folder}")

    # Locate HDF5 files
    pattern = f"*{keyword}*_azimuthal_data.h5" if azimuthal else f"*{keyword}*_radial_data.h5"
    h5_files = glob.glob(os.path.join(target_folder, pattern))

    if not h5_files:
        raise FileNotFoundError(f"No matching HDF5 files found in {target_folder} with pattern {pattern}")

    data_list = []

    for h5_path in h5_files:
        with h5py.File(h5_path, 'r') as hf:
            for key in hf:
                group = hf[key]

                def get_safe_str(name):
                    val = group.get(name)
                    if val is not None:
                        val = val[()]
                        return val.decode() if isinstance(val, bytes) else val
                    return "N/A"

                def get_safe_val(name):
                    val = group.get(name)
                    return val[()] if val is not None else None

                data = {
                    "filename": get_safe_str("filename"),
                    "real_date_time": get_safe_str("real_date_time"),
                    "i0": get_safe_val("i0"),
                    "i1": get_safe_val("i1"),
                    "Photod": get_safe_val("Photod")
                }

                if azimuthal:
                    data.update({
                        "q": group["q"][()],
                        "I": group["I"][()],
                        "error": group["error"][()]
                    })
                else:
                    data.update({
                        "chi": group["chi"][()],
                        "I": group["I"][()]
                    })

                data_list.append(data)

    return data_list


## write HDF5 file

In [ ]:
import h5py
import numpy as np

def write_sorted_data_to_h5(sorted_data, output_file, mode='unknown'):
    """
    Write sorted peak data into an HDF5 file.

    Parameters:
    -----------
    sorted_data : list of tuples
        Each tuple should contain (peak_intensity, x_array, I_array, filename, scan_number).
        Missing values will be replaced with default 0 or empty array.
    output_file : str
        Full path where the HDF5 file will be saved.
    mode : str, optional
        Type or mode to store in the dataset ('radial', 'azimuthal', etc.). Default is 'unknown'.
    """
    with h5py.File(output_file, 'w') as hf:
        for idx, entry in enumerate(sorted_data):
            # Fill missing values with defaults
            peak = entry[0] if len(entry) > 0 else 0
            x = np.array(entry[1]) if len(entry) > 1 else np.array([0])
            I = np.array(entry[2]) if len(entry) > 2 else np.array([0])
            filename = str(entry[3]) if len(entry) > 3 else 'unknown'
            scan_number = entry[4] if len(entry) > 4 else 0

            group = hf.create_group(f'data_{idx}')
            group.create_dataset('x', data=x)
            group.create_dataset('I', data=I)
            group.create_dataset('filename', data=filename)
            group.create_dataset('peak_intensity', data=peak)
            group.create_dataset('type', data=str(mode))
            group.create_dataset('scan_number', data=scan_number)
            group.create_dataset('background_filename', data=filename)
            group.create_dataset('background_file', data=filename)

    print(f"✅ Successfully wrote {len(sorted_data)} entries to '{output_file}'.")



## subtract function v2

In [ ]:
import os
from matplotlib import scale
import numpy as np
import matplotlib.pyplot as plt
import h5py

def subtract_background_v2(
    base_path,
    samp_folder,
    bkg_folder,
    keyword,
    background_keyword,
    subtract_azimuthal=True,
    subtract_chi=False,
    scaling_factor=1.0,
    plot=True,
    use_sample_as_background=False,
    background_filename=None,
    match_partial=False
):
    def process_subtraction(sample_data, background_data, x_key, save_suffix):
        subtracted_data = []

        for samp, bg in zip(sample_data, background_data):
            print(f"Subtracting background from sample:")
            print(f"  Sample file     : {samp['filename']}")
            print(f"  Background file : {bg['filename']}\n")

            x = samp[x_key]
            I_sample = np.array(samp["I"])


            if subtract_azimuthal and "error" in samp:
                err = samp["error"]
            else:
                err = None

            # find the average of I values in the range of 8-10 q range
            if x_key == "q":
                q_range = (x >= 4) & (x <= 7)
                if np.any(q_range):
                    avg_I = np.mean(I_sample[q_range])
                    avg_I_bg = np.mean(bg["I"][q_range])
                    print(f"Average I in q range 8-10: {avg_I}")
                    print(f"Average I in background q range 8-10: {avg_I_bg}")
                    if avg_I_bg > 0:
                        scaling_factor = 0.98*(avg_I / avg_I_bg)
                        print(f"Scaling factor for azimuthal data: {scaling_factor}")
                    else:
                        print("Warning: Average I in background is zero, using default scaling factor.")
                        scaling_factor = 1.0
                else:
                    print("Warning: No valid q range found for scaling.")
                    scaling_factor = 1.0
            
            else:
                # For chi data, we can use a different range or method
                chi_range = (x >= -3) & (x <= 0)
                if np.any(chi_range):
                    avg_I = np.mean(I_sample[chi_range])
                    avg_I_bg = np.mean(bg["I"][chi_range])
                    print(f"Average I in chi range 8-10: {avg_I}")
                    print(f"Average I in background chi range 8-10: {avg_I_bg}")
                    if avg_I_bg > 0:
                        scaling_factor = 0 #*(avg_I / avg_I_bg)
                        print(f"Scaling factor for chi data: {scaling_factor}")
                    else:
                        print("Warning: Average I in background is zero, using default scaling factor.")    
                        # Use the average of the sample data for scaling
                        #print(f"Scaling factor for azimuthal data: {scaling_factor}")
                        scaling_factor = 0
                else:
                    print("Warning: No valid q range found for scaling.")
                    scaling_factor = 0
            I_background = np.array(bg["I"]) * scaling_factor

            real_date_time = samp["real_date_time"]
            i0 = samp["i0"]
            i1 = samp["i1"]
            Photod = samp["Photod"]

            I_subtracted = I_sample - I_background


            subtracted_data.append({
                x_key: x,
                "I": I_subtracted,
                "filename": samp["filename"],
                "err": err,
            })


            if plot:
                plt.figure(figsize=(6, 4))
                plt.plot(x, I_sample, '-', markersize=1, label='Sample')
                plt.plot(x, I_background, '-', markersize=1, label='Background')
                plt.plot(x, I_subtracted, '-', markersize=1, label='Subtracted')
                plt.xlabel(x_key + ' [1/nm]' if x_key == "q" else 'Chi [degrees]')
                plt.ylabel('Intensity [a.u.]')
                plt.title(f"Subtraction: {samp['filename']}")
                set_plot_style(plt.gca(), 20, x_key + ' [1/nm]' if x_key == "q" else 'Chi [degrees]', 'Intensity [a.u.]')
                plt.legend()
                plt.tight_layout()
                plt.show()

        # Save to HDF5
        save_folder = os.path.join(base_path)
        output_folder = os.path.join(save_folder, 'Subtracted_Data')
        os.makedirs(output_folder, exist_ok=True)
        output_file = os.path.join(output_folder, f'subtracted_{keyword}_{save_suffix}.h5')
        if os.path.exists(output_file):
            print(f"File already exists: {output_file}. Overwriting.")
        else:
            print(f"Creating new file: {output_file}")
        
        print(f"Saving subtracted data to {output_file}")

        with h5py.File(output_file, 'w') as hf:
            for idx, data in enumerate(subtracted_data):
                group = hf.create_group(f'data_{idx}')
                
                # Determine x_key: either 'q' or 'chi'
                x_key = 'q' if 'q' in data else 'chi'
                
                # Save both 'x' (generic) and specific axis ('q' or 'chi')
                group.create_dataset('x', data=data[x_key])
                group.create_dataset(x_key, data=data[x_key])
                # Replace 'azimuthal' with this in HDF5 writing section:
                if x_key == 'q' and 'err' in data:
                    group.create_dataset('error', data=data['err'])
                
                group.create_dataset('I', data=data["I"])
                group.create_dataset('filename', data=data["filename"])
                group.create_dataset('scaling_factor', data=scaling_factor)
                group.create_dataset('type', data=save_suffix)
                group.create_dataset('background_filename', data=background_filename if use_sample_as_background else background_keyword)
                group.create_dataset('background_file', data=background_data[idx]['filename'])
                group.create_dataset('real_date_time', data=(real_date_time))
                group.create_dataset('i0', data=np.float32(i0) if i0 is not None else np.nan)
                group.create_dataset('i1', data=np.float32(i1) if i1 is not None else np.nan)
                group.create_dataset('Photod', data=np.float32(Photod) if Photod is not None else np.nan)

        return subtracted_data

    all_results = {}

    for mode, x_key, flag in [
        ("azimuthal_data", "q", subtract_azimuthal),
        ("radial_data", "chi", subtract_chi)
    ]:
        if not flag:
            continue

        # Read sample data
        sample_data = read_h5_data(base_path, samp_folder, keyword, azimuthal=(x_key == "q"))
        sample_data = sorted(sample_data, key=lambda x: x['filename'])

        if use_sample_as_background:
            if background_filename is None:
                raise ValueError("Please specify 'background_filename' when using a sample file as background.")

            selected_bg = None
            for data in sample_data:
                fname = data["filename"]
                if (fname == background_filename) or (match_partial and background_filename in fname):
                    selected_bg = data
                    break

            if selected_bg is None:
                print("Available filenames:")
                for d in sample_data:
                    print(" -", d["filename"])
                raise ValueError(f"No sample file matches background_filename='{background_filename}'.")

            background_data = [selected_bg] * len(sample_data)
            print(f"\n✅ Using sample file as background: {selected_bg['filename']} for {mode}\n")
        else:
            background_data = read_h5_data(base_path, bkg_folder, background_keyword, azimuthal=(x_key == "q"))
            background_data = sorted(background_data, key=lambda x: x['filename'])

            if len(sample_data) != len(background_data):
                # use first background file for all
                print(f"Sample data length: {len(sample_data)}")
                print(f"Background data length: {len(background_data)}")
                print("Using first background file for all samples.")
                background_data = [background_data[0]] * len(sample_data)
                #raise ValueError(f"Sample and background data lengths do not match for {mode} subtraction.")

        all_results[mode] = process_subtraction(sample_data, background_data, x_key, save_suffix=mode)

    return all_results





# poni file for 2D image creation

In [ ]:
from math import e
import numpy as np
import os

# Base paths and file definitions
#base_path = '/Users/akmaurya/Desktop/20240628/in_situ'  


base_path_2D = '/Volumes/SSD1/RawData1/Redesigned_Plastics/Dec2024/'
#poni_file = os.path.join(base_path_2D, 'poni/poni_exsitu/LaB6_poni_New_01.poni')
poni_file = os.path.join(base_path_2D, 'poni/poni_insitu/LaB6_insitu_01.poni')
#mask_file = os.path.join(base_path, 'poni/poni_exsitu/exsitu_mask_new_09.edf')
mask_file = os.path.join(base_path_2D, 'poni/poni_insitu/mask_insitu_new_01.edf')
output_base_path_2D = base_path_2D  # Base path for the output directory structure

ai = pyFAI.load(poni_file) 
print (f"ai: {ai}")


from tkinter import font
import numpy as np
import matplotlib.pyplot as plt
import pyFAI

import numpy as np
import pyFAI
import os
import matplotlib.pyplot as plt

def plot_2d_images(image, poni_file, output_folder, tif_file, azm_range, q_range_2D,mode='azimuthal', vmin=None, vmax=None):

    # Load poni file to extract calibration parameters
    ai = pyFAI.load(poni_file)
    pixel_size = ai.pixel1  # Assuming square pixels
    detector_distance = ai.dist

    #print(ai)

    #beamx = 1028.789
    #beamy = 1126.865

    beamx = ai.getFit2D()['centerX']
    beamy = ai.getFit2D()['centerY']


    #print(f"beamx: {beamx}, beamy: {beamy}")

    wavelength = ai.wavelength

    # Convert SAXS to q-space coordinates
    x_pixels = image.shape[1]
    y_pixels = image.shape[0]
    x_coords = np.arange(x_pixels) - beamx
    y_coords = np.arange(y_pixels) - beamy
    xx, yy = np.meshgrid(x_coords, y_coords)

    qx = 1e-9 * 2 * np.pi / wavelength * np.sin(pixel_size * xx / detector_distance)
    qy = 1e-9 * 2 * np.pi / wavelength * np.sin(pixel_size * yy / detector_distance)

    # Calculate azimuthal angle (in degrees) for each pixel relative to the beam center
    azimuthal_angles = np.degrees(np.arctan2(qy, qx))
    
    # Create a mask for the azimuthal angle range
    mask_azm = (azimuthal_angles >= azm_range[0]) & (azimuthal_angles <= azm_range[1])

    # Create a mask for the q range
    mask_q = (np.sqrt(qx**2 + qy**2) >= q_range_2D[0]) & (np.sqrt(qx**2 + qy**2) <= q_range_2D[1])

    # Plot the 2D image with the sector highlighted
    fig, ax = plt.subplots(figsize=(6, 6))
    # set nans to 0
    image = np.nan_to_num(image, nan=0.0)
    # Normalize the image  with sum of intensity
    image = 10000000* (image / np.sum(image))

    
    #qx = -qx
    #qy = -qy  # Invert y-axis for correct orientation
    # take log of the intensity 

    # Apply logarithmic transformation to the intensity values
    #image = np.log1p(image)  # log1p is used to handle zero values safely (log(1 + x))

    
    im_avg = ax.pcolormesh(qx, qy, (image), cmap='jet', vmin=vmin, vmax=vmax, shading='auto', rasterized=True,zorder=0)
    ax.invert_yaxis( )
    # Correct the labels: flip the sign of qy for display
    ax.set_yticks(np.flip(ax.get_yticks()))  # Flip y-ticks
        # Change the sign of the y-axis labels
    y_ticks = ax.get_yticks()
    ax.set_yticklabels([-int(tick) for tick in y_ticks])  # Convert to integers and negate


    x_ticks = np.arange(-20, 21, 10)  # Adjust this range as needed
    ax.set_xticks(x_ticks)
    ax.set_xticklabels([str(abs(tick)) for tick in x_ticks])  # Display only positive integer labels for x-axis




    # flip the axis sign up positive

    
    
    ax.set_aspect('equal')
    
    # Overlay the sector by masking the area outside azimuthal range
    if mode == 'azimuthal':

        #ax.contourf(qx, qy, mask_azm, levels=[0.5, 1], colors='white', alpha=0.3,zorder=1)  # hatches=['//']
        # Semi-transparent fill (distinct from background and wedge)
        #ax.contourf(qx, qy, mask_azm, levels=[0.5, 1], colors='green', alpha=0.1, zorder=3)
        # Dotted black contour edge
        ax.contour(qx, qy, mask_azm, levels=[0.5], colors='white', linewidths=1.5,
                linestyles='dotted', zorder=4)
    
    if mode == 'radial':
        # Overlay the sector by masking the area outside q range
        #ax.contourf(qx, qy, mask_q, levels=[0.5, 1], colors='white', alpha=0.3,zorder=1)  # hatches=['//']

        # Semi-transparent fill (distinct from background and wedge)
        #ax.contourf(qx, qy, mask_q, levels=[0.5, 1], colors='green', alpha=0.1, zorder=3)

        # Dotted black contour edge
        ax.contour(qx, qy, mask_q, levels=[0.5], colors='white', linewidths=1.5,
                linestyles='dotted', zorder=4)

    cbar_avg = fig.colorbar(im_avg, ax=ax, shrink=.8, label='Intensity [a.u.]')
    cbar_avg.ax.tick_params(labelsize=20)
    cbar_avg.ax.yaxis.label.set_size(20)
    
    ax.set_xlabel(r"$q_x$ [nm$^{-1}$]", fontsize=20)
    ax.set_ylabel(r"$q_y$ [nm$^{-1}$]", fontsize=20)
    ax.set_title(f"{f'{os.path.splitext(tif_file)[0]}_2D'}", fontsize=14, y=1.05)
    
    ax.tick_params(axis='both', which='major', labelsize=20, width=1.5)

    # Save and show the figure
    # creat a Figures folder with output_folder
    output_folder = os.path.join(output_folder, "Figures")
    os.makedirs(output_folder, exist_ok=True)
    # create the folder if it does not exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    # save the figure
    plt.savefig(f"{output_folder}/{f'{os.path.splitext(tif_file)[0]}'}.png", dpi=300, bbox_inches='tight')
    #plt.close()
    plt.show()



## 2D image creation

In [ ]:
import os
import glob
from xml.etree.ElementTree import TreeBuilder
import fabio
import imageio
import numpy as np
from tqdm import tqdm


def extract_folder_and_filenames(selected_files):
    """
    Extracts folder names and filenames based on the selected filenames.
    
    Folder name is the part after 'thresh1_' up to '_s' in the filename.
    Filename is kept as-is.
    
    Parameters:
        selected_files (list of str): List of filenames.
        
    Returns:
        foldernames (list of str): List of folder names extracted.
        filenames (list of str): List of filenames (same as input).
    """
    foldernames = []
    filenames = []
    
    for file in selected_files:
        try:
            # Remove prefix 'thresh1_' and split at '_s'
            name_part = file.split('thresh1_')[1]
            foldername = name_part.split('_s000')[0]
            foldernames.append(foldername)
            filenames.append(file)
        except IndexError:
            print(f"Warning: Could not process file {file}")
            continue
    
    return foldernames, filenames





def average_selected_tif_images(base_path_2D, samp_folder, selected_files, bkg_files, poni_file, mask_file, output_base_path_2D, mode, 
                                vmin=None, vmax=None, q_range_2D=(3, 35), azm_range=(-105, -75),plot=False):
    """
    Average selected .tif images after background subtraction, apply mask, and save averaged image and plot.
    """

    # Load the mask properly
    mask = fabio.open(mask_file).data

    # Prepare to collect images
    image_list = []

    # Extract foldernames and filenames separately
    foldernames, filenames = extract_folder_and_filenames(selected_files)
    bkg_folders, bkg_filenames = extract_folder_and_filenames(bkg_files)

    # Track failed files
    failed_files = []

    # Loop over selected files
    for foldername, filename, bkg_folder, bkg_filename in tqdm(zip(foldernames, filenames, bkg_folders, bkg_filenames), total=len(filenames), desc="Processing images"):
        print(f"Processing {foldername}/{filename} (background: {bkg_filename})...")

        tif_file_path = os.path.join(base_path_2D, 'P3HB', foldername, 'Threshold 1', filename)
        bkg_file_path = os.path.join(base_path_2D, 'P3HB', bkg_folder, 'Threshold 1', bkg_filename)

        try:
            img_sample = imageio.v3.imread(tif_file_path)
            img_bkg = imageio.v3.imread(bkg_file_path)
        except Exception as e:
            print(f"Error reading {tif_file_path} or {bkg_file_path}: {e}")
            failed_files.append((filename, bkg_filename))
            continue

        img = img_sample - 0.95*img_bkg  # Subtract background
        img[mask == 1] = 0           # Apply mask
        image_list.append(img)

        if plot:
            try:
                save_folder = os.path.join(output_base_path_2D, samp_folder, 'Individual_2D')
                os.makedirs(save_folder, exist_ok=True)
                plot_2d_images(img, poni_file, save_folder, os.path.basename(filename),
                               vmin=vmin, vmax=vmax, azm_range=azm_range,q_range_2D=q_range_2D,mode=mode)
            except Exception as plot_error:
                print(f"Error plotting {filename}: {plot_error}")

    # After loop: average the images
    if image_list:
        avg_image = np.mean(image_list, axis=0)
        avg_image[mask == 1] = 0  # Apply mask again just to be sure

        avg_save_folder = os.path.join(output_base_path_2D, samp_folder, 'Average_2D')
        os.makedirs(avg_save_folder, exist_ok=True)

        avg_filename = f'{foldername}_{mode}_average.tiff'
        avg_save_path = os.path.join(avg_save_folder, avg_filename)

        imageio.imwrite(avg_save_path, avg_image.astype(np.float32))
        print(f"Averaged image saved at {avg_save_path}")

        if plot:
            try:
                plot_2d_images(avg_image, poni_file, avg_save_folder, f'{foldername}_{mode}_average.png',
                               vmin=vmin, vmax=vmax, azm_range=azm_range,q_range_2D=q_range_2D,mode=mode)
            except Exception as plot_avg_error:
                print(f"Error plotting average image: {plot_avg_error}")

    else:
        print("No images were collected for averaging.")

    if failed_files:
        print(f"Warning: {len(failed_files)} files failed to load:")
        for sample, bkg in failed_files:
            print(f"  Sample: {sample} | Background: {bkg}")


# Subtract background 

## insitu

## LDPE

In [ ]:

base_path = '/Volumes/SSD1/RawData1/Redesigned_Plastics/Dec2024/OneD_integrated_WAXS_01/insitu/'
samp_folder = 'LDPE'
match_partial = True
scaling_factor = 1.0
plot = True

# List of (keyword, background_filename, subtract_azimuthal, subtract_chi, use_sample_as_background, bkg_folder, background_keyword)
samples = [

   #('Run33_f_P3HB_07_50rpm_80C_01', 's00000', True, True, True,'', ''),

    
    ('insitu_Run2_LDPE_5mmmin_01_Scan00001', '', True, True, False, 'Air', 'insitu_Run11_air_with_TS600_01'),
    ('insitu_Run16_LDPE_film_3mmmin_01_Scan00001', '', True, True, False, 'Air', 'insitu_Run11_air_with_TS600_01'),



  
]

for keyword, background_filename, subtract_azimuthal, subtract_chi, use_sample_as_background, bkg_folder, background_keyword in samples:
    subtract_background_v2(
        base_path=base_path,
        samp_folder=samp_folder,
        bkg_folder=bkg_folder,
        keyword=keyword,
        background_keyword=background_keyword,
        subtract_azimuthal=subtract_azimuthal,
        subtract_chi=subtract_chi,
        scaling_factor=scaling_factor,
        plot=plot,
        use_sample_as_background=use_sample_as_background,
        background_filename=background_filename,
        match_partial=match_partial
    )


## PHPD

In [ ]:
base_path = '/Volumes/SSD1/RawData1/Redesigned_Plastics/Dec2024/OneD_integrated_WAXS_01/insitu/'
samp_folder = 'PHPD'
match_partial = True
scaling_factor = 1.0
plot = False

# List of (keyword, background_filename, subtract_azimuthal, subtract_chi, use_sample_as_background, bkg_folder, background_keyword)
samples = [
    #heated
    ##('Run32_f_PET_10_50rpm_3x_cold_draw__120C_01', 's00000', True, True, True,'', ''),  # scaling = 0.93
 
    ('insitu_Run6_PHPD_film_base_5mmmin_01', '', True, True, False, 'Air', 'insitu_Run11_air_with_TS600_01'),
    ('insitu_Run8_PHPD_film_base_3mmmin_01', '', True, True, False, 'Air', 'insitu_Run11_air_with_TS600_01'),
    ('insitu_Run9_PHPD_MBfilm_3mmmin_01', '', True, True, False, 'Air', 'insitu_Run11_air_with_TS600_01'),
    
]

#keywords= ['cryomill_PET_powder_01_Scan00011','Run17_rep_f_PET_02_50rpm_as_spun_01_Scan00001','Run18_f_PET_03_100rpm_as_spun_01_Scan00001','Run19_f_PET_04_200rpm_as_spun_01_Scan00001','Run20_f_PET_05_400rpm_as_spun_01_Scan00001','Run21_f_PET_06_600rpm_as_spun_01_Scan00001']



for keyword, background_filename, subtract_azimuthal, subtract_chi, use_sample_as_background, bkg_folder, background_keyword in samples:
    subtract_background_v2(
        base_path=base_path,
        samp_folder=samp_folder,
        bkg_folder=bkg_folder,
        keyword=keyword,
        background_keyword=background_keyword,
        subtract_azimuthal=subtract_azimuthal,
        subtract_chi=subtract_chi,
        scaling_factor=scaling_factor,
        plot=plot,
        use_sample_as_background=use_sample_as_background,
        background_filename=background_filename,
        match_partial=match_partial
    )


## P5HV

In [ ]:
base_path = '/Volumes/SSD1/RawData1/Redesigned_Plastics/Dec2024/OneD_integrated_WAXS_01/insitu/'
samp_folder = 'P5HV'
match_partial = True
scaling_factor = 1.0
plot = False

# List of (keyword, background_filename, subtract_azimuthal, subtract_chi, use_sample_as_background, bkg_folder, background_keyword)
samples = [
    #('Run29_f_PLLA_08_50rpm_80C_tension_01', 's00035', True, True, True,'', ''),


    ('insitu_Run10_P5HV_film_3mmmin_01', '', True, True, False, 'Air', 'insitu_Run11_air_with_TS600_01'),
    ('insitu_Run14_P5HV_film_3mmmin_01', '', True, True, False, 'Air', 'insitu_Run11_air_with_TS600_01'),
    ('insitu_Run15_P5HV_film_3mmmin_01', '', True, True, False, 'Air', 'insitu_Run11_air_with_TS600_01'),
    ('insitu_Run17_P5HV_film_5mmmin_01', '', True, True, False, 'Air', 'insitu_Run11_air_with_TS600_01'),
]


for keyword, background_filename, subtract_azimuthal, subtract_chi, use_sample_as_background, bkg_folder, background_keyword in samples:
    subtract_background_v2(
        base_path=base_path,
        samp_folder=samp_folder,
        bkg_folder=bkg_folder,
        keyword=keyword,
        background_keyword=background_keyword,
        subtract_azimuthal=subtract_azimuthal,
        subtract_chi=subtract_chi,
        scaling_factor=scaling_factor,
        plot=plot,
        use_sample_as_background=use_sample_as_background,
        background_filename=background_filename,
        match_partial=match_partial
    )


# plot 1D and 2D

In [ ]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import glob
import re
from collections import defaultdict

def extract_scan_number(filename):
    """Extract scan number from filename like ...Scan00010_....tif → 10"""
    match = re.search(r'Scan(\d+)', filename)
    if match:
        return int(match.group(1))
    return None

def plot_subtracted_data(base_path, samp_folder, keyword, poni_file, mask_file,
                         vmin=0, vmax=15, mode="azimuthal", pattern_name="subtracted",
                         filter_intensity=1000, sort_by='scan', top_n=None,
                         q_range_2D=(3, 35), azm_range=(-180, 0), plot_2D=False):
    subtracted_dir = os.path.join(base_path, 'Subtracted_Data')
    modes = ['azimuthal', 'radial'] if mode == "both" else [mode]

    # Dictionaries to store entries keyed by filename
    azimuthal_entries = {}
    radial_entries = {}
    plt.figure(figsize=(6, 4))

    for current_mode in modes:
        search_pattern = os.path.join(subtracted_dir, f"{pattern_name}_*{keyword}*_{current_mode}_data.h5")
        matching_files = glob.glob(search_pattern)
        if not matching_files:
            raise FileNotFoundError(f"No file matching pattern: {search_pattern}")

        print(f"\nProcessing mode: {current_mode}")
        for subtracted_file in matching_files:
            print(f"Reading: {subtracted_file}")
            with h5py.File(subtracted_file, 'r') as hf:
                keys = sorted(
                    hf.keys(),
                    key=lambda k: hf[k]["filename"][()].decode("utf-8")
                    if isinstance(hf[k]["filename"][()], bytes)
                    else hf[k]["filename"][()]
                )
                for key in keys:
                    group = hf[key]
                    x = np.array(group["x"])
                    I = np.array(group["I"])
                    filename = group["filename"][()].decode("utf-8") if isinstance(group["filename"][()], bytes) else group["filename"][()]
                    scan_number = extract_scan_number(filename)
                    bkg_filename = group["background_file"][()].decode("utf-8") if isinstance(group["background_file"][()], bytes) else group["background_file"][()]
                    if scan_number is None:
                        continue

                    if current_mode == "azimuthal":
                        mask = (x >= 3) & (x <= 45)
                        x_masked = x[mask]
                        I_masked = I[mask]
                        focus_mask = (x_masked >= 5) & (x_masked <= 20)
                    else:  # radial
                        x_masked = x
                        I_masked = I
                        focus_mask = (x_masked >= -100) & (x_masked <= -80)

                    I_focus = I_masked[focus_mask]
                    peak_intensity = np.max(I_focus)
                    entry = (peak_intensity, x_masked, I_masked, filename, scan_number, bkg_filename)

                    plt.plot(x_masked, I_masked, 'o-', markersize=1)
                    #print(mode)
                    set_plot_style(plt.gca(), 20, "q [1/nm]" if current_mode == "azimuthal" else "Chi [degrees]", "Intensity [a.u.]")
                    plt.title(f"{filename}")

                    if current_mode == "azimuthal":
                        azimuthal_entries[filename] = entry
                    else:
                        radial_entries[filename] = entry

                plt.tight_layout()
                plt.savefig(os.path.join(subtracted_dir, f"{pattern_name}_{keyword}_{mode}_scan.png"), dpi=300)
                plt.show()

    # Match filenames present in both azimuthal and radial
    common_filenames = set(azimuthal_entries.keys()) & set(radial_entries.keys())

    filtered_data = {"azimuthal": [], "radial": []}
    for fname in common_filenames:
        az_entry = azimuthal_entries[fname]
        rad_entry = radial_entries[fname]

        az_high = np.max(az_entry[2]) > filter_intensity
        #rad_high = np.max(rad_entry[2]) > filter_intensity
        if az_high:
            print(f"Skipping {fname} due to high intensity in both modes.")
            continue

        filtered_data["azimuthal"].append(az_entry)
        filtered_data["radial"].append(rad_entry)

    



    all_mode_data = filtered_data

    # Selection based on sorting strategy
    selected_data = {}
    for current_mode in modes:
        data = all_mode_data[current_mode]
        if sort_by == 'scan':
            scan_data = defaultdict(list)
            for entry in data:
                scan_data[entry[4]].append(entry)
            selected = [max(entries, key=lambda t: t[0]) for entries in scan_data.values()]
            sorted_data = sorted(selected, key=lambda t: t[4])
        elif sort_by == 'all':
            sorted_data = sorted(data, key=lambda t: t[0], reverse=True)
        else:
            raise ValueError("sort_by must be 'scan' or 'all'")

        if top_n is not None:
            sorted_data = sorted_data[:top_n]

        selected_data[current_mode] = sorted_data

    # Use the same filenames for both radial and azimuthal if mode == "both"
    if mode == "both":
        selected_files = [t[3] for t in selected_data['azimuthal']]
        bkg_files = [t[5] for t in selected_data['azimuthal']]
    else:
        selected_files = [t[3] for t in selected_data[mode]]
        bkg_files = [t[5] for t in selected_data[mode]]

    print(f"{'Sample File':<50}")
    print('-' * 80)
    for sample in (selected_files):
        print(f"{sample:<50}")

    base_path_2D = os.path.dirname(os.path.dirname(os.path.dirname(base_path)))
    output_folder_2D = os.path.join(base_path, 'Subtracted_Data', 'Averaged', '2DFigures')
    os.makedirs(output_folder_2D, exist_ok=True)

    for current_mode in modes:
        average_selected_tif_images(
            base_path_2D=base_path_2D,
            samp_folder=samp_folder,
            selected_files=selected_files,
            bkg_files=bkg_files,
            poni_file=poni_file,
            mask_file=mask_file,
            output_base_path_2D=output_folder_2D,
            vmin=vmin, vmax=vmax,
            mode=current_mode,
            q_range_2D=q_range_2D,
            azm_range=azm_range,
            plot=plot_2D
        )

        # Plotting 1D data
        data = selected_data[current_mode]
        x_ref = data[0][1]
        all_intensities = []

        plt.figure(figsize=(8, 6))
        for i, (peak, x, I, filename, _, bkg) in enumerate(data):
            I_norm = I.copy()  # Make a copy so you don't overwrite I

            '''
            if current_mode == 'radial':
            
                peak_index = np.argmax(I_norm)  # Find the index of the peak in the original array
                peak_x = x[peak_index]     # Get the corresponding x-value at the peak

                if I_norm[peak_index] > 2000:
                    x = x - peak_x - 90
                    #x_ref = x
            '''

            all_intensities.append(I_norm)
            plt.plot(x, I_norm, '-', alpha=0.4, label=os.path.basename(filename) if i < 10 else "")

        avg_I = np.mean(all_intensities, axis=0)
        plt.plot(x_ref, avg_I, 'k-', linewidth=2.5, label='Average')

        filename_avg = os.path.basename(filename).replace('.h5', '_avg.h5')

        set_plot_style(plt.gca(), 20, "q [1/nm]" if current_mode == "azimuthal" else "Chi [degrees]", "Normalized Intensity [a.u.]")

        output_folder = os.path.join(base_path, 'Subtracted_Data', 'Averaged', 'Filtered')
        os.makedirs(output_folder, exist_ok=True)
        output_folder_avg = os.path.join(base_path, 'Subtracted_Data', 'Averaged')

        txt_file = os.path.join(output_folder_avg, f'subtracted_avg_{keyword}_{current_mode}_sortby_{sort_by}_top{len(data) if top_n else "all"}.txt')
        h5_file = os.path.join(output_folder_avg, f'subtracted_avg_{keyword}_{current_mode}_sortby_{sort_by}_top{len(data) if top_n else "all"}.h5')
        png_file = os.path.join(output_folder, f'Subtracted_{keyword}_{current_mode}_sortby_{sort_by}_top{len(data) if top_n else "all"}.png')
        sorted_h5_file = os.path.join(output_folder, f'subtracted_{keyword}_{current_mode}_sortby_{sort_by}_top{len(data) if top_n else "all"}.h5')

        np.savetxt(txt_file, np.column_stack((x_ref, avg_I)), header="q [1/nm] \t Avg Intensity [a.u.]")
        print(f"Saved averaged data to {txt_file}")
        write_sorted_data_to_h5([(0, x_ref, avg_I, filename_avg, 0)], h5_file, mode=current_mode)
        write_sorted_data_to_h5(data, sorted_h5_file, mode=current_mode)
        print(f"Saved sorted data to {sorted_h5_file}")

        plt.title(f'Averaged_{keyword}_{current_mode}_sortby_{sort_by}_top{len(data) if top_n else "all"}')
        plt.tight_layout()
        plt.savefig(png_file, dpi=300)
        plt.show()


In [ ]:
import os
import glob
import h5py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors

def plot_subtracted_data_all(base_path, samp_folder, keyword, poni_file, mask_file,
                             vmin=0, vmax=15, mode="azimuthal", pattern_name="subtracted",
                             q_range_2D=(3, 35), azm_range=(-180, 0), plot_2D=False):
    subtracted_dir = os.path.join(base_path, 'Subtracted_Data')
    modes = ['azimuthal', 'radial'] if mode == "both" else [mode]

    for current_mode in modes:
        search_pattern = os.path.join(subtracted_dir, f"{pattern_name}_*{keyword}*_{current_mode}_data.h5")
        matching_files = glob.glob(search_pattern)
        if not matching_files:
            raise FileNotFoundError(f"No file matching pattern: {search_pattern}")

        print(f"\nProcessing mode: {current_mode}")
        all_data = []

        # For surface plot data collection
        intensity_matrix = []
        x_axis = None

        # Set up plot
        plt.figure(figsize=(6, 4))

        # Colormap setup for line plot
        num_colors = sum(len(h5py.File(f, 'r').keys()) for f in matching_files)
        cmap = cm.get_cmap('viridis', num_colors)
        norm = mcolors.Normalize(vmin=0, vmax=num_colors - 1)

        color_idx = 0

        for subtracted_file in matching_files:
            print(f"Reading: {subtracted_file}")
            with h5py.File(subtracted_file, 'r') as hf:
                keys = sorted(
                    hf.keys(),
                    key=lambda k: hf[k]["filename"][()].decode("utf-8")
                    if isinstance(hf[k]["filename"][()], bytes)
                    else hf[k]["filename"][()]
                )
                for key in keys:
                    group = hf[key]
                    x = np.array(group["x"])
                    I = np.array(group["I"])
                    filename = group["filename"][()].decode("utf-8") if isinstance(group["filename"][()], bytes) else group["filename"][()]
                    bkg_filename = group["background_file"][()].decode("utf-8") if isinstance(group["background_file"][()], bytes) else group["background_file"][()]

                    if current_mode == "azimuthal":
                        mask = (x >= 0) & (x <= 45)
                        x_masked = x[mask]
                        I_masked = I[mask]
                    else:
                        x_masked = x
                        I_masked = I

                    all_data.append((x_masked, I_masked, filename, bkg_filename))
                    intensity_matrix.append(I_masked)
                    if x_axis is None:
                        x_axis = x_masked  # Use x-axis from the first entry

                    color = cmap(norm(color_idx))
                    plt.plot(x_masked, I_masked, '-', alpha=0.8, color=color, label=os.path.basename(filename))
                    color_idx += 1

        # Line plot
        set_plot_style(plt.gca(), 20, "q [1/nm]" if current_mode == "azimuthal" else "Chi [degrees]", "Intensity [a.u.]")
        plt.title(f"{keyword} - {current_mode} - All Scans")
        plt.tight_layout()
        plt.savefig(os.path.join(subtracted_dir, f"{pattern_name}_{keyword}_{current_mode}_ALL.png"), dpi=300)
        plt.show()

        # Surface Plot
        intensity_matrix = np.array(intensity_matrix)
        if intensity_matrix.ndim == 2:
            plt.figure(figsize=(7, 5))
            extent = [x_axis[0], x_axis[-1], 0, intensity_matrix.shape[0]]
            aspect = 'auto'

            plt.imshow(intensity_matrix, extent=extent, origin='lower',
                       aspect=aspect, cmap='viridis', vmin=np.min(intensity_matrix),
                       vmax=np.max(intensity_matrix))
            plt.colorbar(label="Intensity [a.u.]")
            plt.xlabel("q [1/nm]" if current_mode == "azimuthal" else "Chi [degrees]")
            plt.ylabel("Scan index")
            plt.title(f"Surface Plot - {keyword} - {current_mode}")
            plt.tight_layout()
            plt.savefig(os.path.join(subtracted_dir, f"{pattern_name}_{keyword}_{current_mode}_SURFACE.png"), dpi=300)
            plt.show()

        # Optional 2D plotting (external function)
        if plot_2D:
            base_path_2D = os.path.dirname(os.path.dirname(os.path.dirname(base_path)))
            output_folder_2D = os.path.join(base_path, 'Subtracted_Data', '2DFigures')
            os.makedirs(output_folder_2D, exist_ok=True)

            selected_files = [t[2] for t in all_data]
            bkg_files = [t[3] for t in all_data]

            average_selected_tif_images(
                base_path_2D=base_path_2D,
                samp_folder=samp_folder,
                selected_files=selected_files,
                bkg_files=bkg_files,
                poni_file=poni_file,
                mask_file=mask_file,
                output_base_path_2D=output_folder_2D,
                vmin=vmin, vmax=vmax,
                mode=current_mode,
                q_range_2D=q_range_2D,
                azm_range=azm_range,
                plot=True
            )


## LDPE

In [ ]:
# P3HB dataset processing
base_path = '/Volumes/SSD1/RawData1/Redesigned_Plastics/Dec2024/OneD_integrated_WAXS_01/insitu'
samp_folder = 'LDPE'

print(f"Processing {samp_folder}...")

samples = [
    ("insitu_Run2_LDPE_5mmmin_01_Scan00001"),
    ("insitu_Run16_LDPE_film_3mmmin_01_Scan00001"),

]
for keyword in samples:
    plot_subtracted_data_all(
        base_path=base_path,
        samp_folder=samp_folder,
        poni_file=poni_file,
        mask_file=mask_file,
        keyword=keyword,
        mode="both",
        
        q_range_2D=(14, 16), 
        azm_range=(-45, 45),
        
        plot_2D=True
    )




## PHPD

In [ ]:
# P3HB dataset processing
base_path = '/Volumes/SSD1/RawData1/Redesigned_Plastics/Dec2024/OneD_integrated_WAXS_01/insitu'
samp_folder = 'PHPD'

print(f"Processing {samp_folder}...")

samples = [
    'insitu_Run6_PHPD_film_base_5mmmin_01', 
    'insitu_Run8_PHPD_film_base_3mmmin_01',
    'insitu_Run9_PHPD_MBfilm_3mmmin_01', 

]
for keyword in samples:
    plot_subtracted_data_all(
        base_path=base_path,
        samp_folder=samp_folder,
        poni_file=poni_file,
        mask_file=mask_file,
        keyword=keyword,
        mode="both",
        
        q_range_2D=(14, 16), 
        azm_range=(-45, 45),
        
        plot_2D=True
    )




## P5HV

In [ ]:
# P3HB dataset processing
base_path = '/Volumes/SSD1/RawData1/Redesigned_Plastics/Dec2024/OneD_integrated_WAXS_01/insitu'
samp_folder = 'P5HV'

print(f"Processing {samp_folder}...")

samples = [
     'insitu_Run10_P5HV_film_3mmmin_01',
    'insitu_Run14_P5HV_film_3mmmin_01', 
    'insitu_Run15_P5HV_film_3mmmin_01',
    'insitu_Run17_P5HV_film_5mmmin_01', 

]
for keyword in samples:
    plot_subtracted_data_all(
        base_path=base_path,
        samp_folder=samp_folder,
        poni_file=poni_file,
        mask_file=mask_file,
        keyword=keyword,
        mode="both",
        
        q_range_2D=(14, 16), 
        azm_range=(-45, 45),
        
        plot_2D=True
    )




# Plotting- 1D

## Azimuthal integration plot

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import fnmatch
from matplotlib.cm import get_cmap

def plot_filtered_files(folder_path, patterns, xlim=(4, 20), colormap_name='viridis'):
    """
    Plot data from text files in a given folder, filtered by wildcard patterns in filenames.
    All files are plotted using a smooth color transition from a colormap.

    Parameters:
    - folder_path (str): Path to the folder containing the .txt files
    - patterns (list): List of wildcard patterns to filter filenames
    - xlim (tuple): x-axis limits for the plot
    - colormap_name (str): Name of matplotlib colormap
    """

    plt.figure(figsize=(6.5, 6), dpi=300)
    plt.clf()
    plt.ion()

    matched_files = []
    for pattern in patterns:
        for f in sorted(os.listdir(folder_path)):
            if (
                f.endswith('.txt') and
                'azimuthal' in f and
                not f.startswith('._') and
                fnmatch.fnmatch(f, pattern)
            ):
                matched_files.append(f)

    # Apply color transition using the colormap
    cmap = get_cmap(colormap_name)
    colors = cmap(np.linspace(0, 1, len(matched_files)))

    for idx, filename in enumerate(matched_files):
        file_path = os.path.join(folder_path, filename)

        try:
            q_values = []
            intensity_values = []

            with open(file_path, 'r') as f:
                for line in f:
                    if (
                        line.startswith('#') or
                        line.strip() == '' or
                        'dtype' in line or
                        'q:' in line or
                        'intensity:' in line
                    ):
                        continue

                    parts = line.strip().split()
                    if len(parts) >= 2:
                        q_values.append(float(parts[0]))
                        intensity_values.append(float(parts[1]))

            # Create label
            label = filename.split('.')[0]
            label = label.replace('thresh1_', '').split('_Scan')[0]
            label = label.split('f_')[-1] if 'f_' in label else label
            label = label.split('_azimuthal')[0] if 'azimuthal' in label else label
            label = label.replace('P3HB', 'PHA-3B')
            label = label.replace('rep_', '').replace('_01', '')
            label = label.replace('subtracted_avg_', '')

            # Normalize intensity
            intensity_values = np.array(intensity_values)
            intensity_values = 10000 * intensity_values / np.sum(intensity_values)

            plt.plot(q_values, intensity_values, '-o', label=label, markersize=2, color=colors[idx])

        except Exception as e:
            print(f"Failed to read {file_path}: {e}")

    plt.xlabel('q [nm⁻¹]')
    plt.ylabel('Intensity [a.u.]')
    set_plot_style(plt.gca(), 20, 'q [nm⁻¹]', 'Intensity [a.u.]')
    plt.xlim(xlim)
    plt.ylim(0, )
    plt.legend(loc='upper right', fontsize=12, frameon=True)
    plt.tight_layout()
    plt.show()





folder = '/Volumes/SSD1/RawData1/Redesigned_Plastics/Dec2024/OneD_integrated_WAXS_01/P3HB/Subtracted_Data/Averaged'
#patterns = ['*PLLA_*s00010*', '*PLLA*_s000014*']
patterns = ['*cryomill_P3HB_powder*_scan*',
 '*avg_P3HB_*_50RPM*scan*',
 '*avg_P3HB_*_100RPM*scan*',
 '*avg_P3HB_*_200RPM*scan*',
 '*avg_P3HB_*_400RPM*scan*',
 '*avg_P3HB_*_600RPM*scan*'
]
#avg_P3HB_03_100RPM_rep

plot_filtered_files(folder, patterns, xlim=(5, 20))


patterns = ['*cryomill_PLLA_powder*',
 '*PLLA_08_50rpm_80C_tension*',
    '*PLLA_07_50rpm_80C_free*',]
plot_filtered_files(folder, patterns, xlim=(11, 13))

patterns = ['*cryomill*PLLA*',
    '*PLLA*50rpm*as_spun*',
    '*PLLA*100rpm*as_spun*',
    '*PLLA*200rpm*as_spun*',
    '*PLLA*400rpm*as_spun*',
    '*PLLA*600rpm*as_spun*',
    
]


plot_filtered_files(folder, patterns, xlim=(11, 13))


patterns = ['*cryomill_P3HB_powder*',
 '*P3HB_07_50rpm_80C_*scan*',
    '*P3HB_08_50rpm_3xcold_draw_*scan*',
    '*P3HB_09_50rpm_3xcolddraw_80C_*scan*',

]
plot_filtered_files(folder, patterns, xlim=(5, 20))


patterns = ['*cryomill_PET_powder*',
 '*avg_PET_*_50rpm*',
 '*avg_PET_*_100rpm*',
 '*avg_PET_*_200rpm*',
 '*avg_PET_*_400rpm*',
 '*avg_PET_*_600rpm*',
    '*PET_07_400rpm_3xcolddraw*'
 
]
plot_filtered_files(folder, patterns, xlim=(5, 25))


patterns = ['*cryomill_PET_powder*',
 '*PET_10_50rpm_3x_cold_draw*',
 '*PET_09_50rpm_3x_cold_draw*',
 '*PET_08_50rpm_120C*',
 '*PET_07_400rpm_3xcolddraw*',
]
plot_filtered_files(folder, patterns, xlim=(5, 25))

patterns = [#'*cryomill_PET_powder*',
 '*f_PET_Fiber1_Ozge_rep_01*',
 #'*f_PET_Fiber1_Ozge_01*',
 '*f_PET_Fiber2_Ozge_01*',
 '*f_PET_Fiber3_Ozge_01*',
 '*f_PET_Fiber1_Ozge_10min_methanolysis_01*',
]


plot_filtered_files(folder, patterns, xlim=(5, 25))

In [ ]:
import os
import fnmatch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def read_and_plot_chi_intensity(folder_path, patterns, normalize=False, xlim=(-180, -0), ylim=None, figsize=(6.5, 6), dpi=300):
    """
    Plot Chi vs Intensity data using Viridis colormap and styled formatting.
    
    Parameters:
    - folder_path (str): Folder containing .txt files
    - patterns (list of str): Wildcard patterns to match filenames
    - normalize (bool): Normalize intensity if True
    - xlim (tuple): x-axis limits
    - ylim (tuple or None): y-axis limits
    - figsize (tuple): Figure size
    - dpi (int): Plot DPI
    """
    plt.figure(figsize=figsize, dpi=dpi)
    plt.clf()
    plt.ion()

    all_labels = []
    all_data = []

    for filename in sorted(os.listdir(folder_path)):
        if (
            filename.endswith('.txt')
            and 'radial' in filename
            and not filename.startswith('._')
            and any(fnmatch.fnmatch(filename, pattern) for pattern in patterns)
        ):
            file_path = os.path.join(folder_path, filename)

            chi, intensity = [], []
            try:
                with open(file_path, 'r') as file:
                    for line in file:
                        if line.strip() == '' or line.startswith('#'):
                            continue
                        parts = line.strip().split()
                        if len(parts) >= 2:
                            chi.append(float(parts[0]))
                            intensity.append(float(parts[1]))

                if not chi:
                    print(f"No data in {filename}, skipping.")
                    continue

                chi = np.array(chi)
                intensity = np.array(intensity)



                # subtract a constant   basleine bu taking average of 10 lowest intesity values in the chi range of -120 to -20
                mask = (chi >= -180) & (chi <= -0)
                intensity = intensity[mask]
                chi = chi[mask]
                #if len(intensity[mask]) > 10:
                    #baseline = np.mean(intensity[mask][:10])
                    #intensity -= baseline
                #intensity[intensity < 0] = 0  # Set negative values to zero

                # Exclude bad chi range

                

                if normalize:
                    intensity = 10000 * intensity / np.sum(intensity)
                
                mask = (chi <= -80) | (chi >= -70)
                chi = chi[mask]
                intensity = intensity[mask]

                label = filename.split('.')[0].replace('thresh1_', '')
                label = label.split('_radial')[0]
                label = label.split('_avg_')[1] if '_avg_' in label else label
                label = label.split('_01_Scan')[0]
                label = label.replace('rep_', '').replace('_01', '')
                if 'f_' in label:
                    label = label.split('f_')[1]
                if 'P3HB' in label:
                    label = label.replace('P3HB', 'PHA-3B')

                all_labels.append(label)
                all_data.append((chi, intensity))

            except Exception as e:
                print(f"Error in {file_path}: {e}")
    
    cmap = cm.get_cmap('viridis', len(all_data))  # Viridis colormap

    for i, (label, (chi, intensity)) in enumerate(zip(all_labels, all_data)):
        color = cmap(i)
        plt.plot(chi, intensity, '-o', label=label, color=color, markersize=2)

    plt.xlim(*xlim)
    if ylim:
        plt.ylim(*ylim)

    plt.xlabel('Chi [degrees]')
    plt.ylabel('Intensity [a.u.]')
    set_plot_style(plt.gca(), 20, 'Chi [degrees]', 'Intensity [a.u.]')
    plt.legend()
    plt.tight_layout()
    plt.show()




folder = '/Volumes/SSD1/RawData1/Redesigned_Plastics/Dec2024/OneD_integrated_WAXS_01/P3HB/Subtracted_Data/Averaged'
#patterns = ['*PLLA_*s00010*', '*PLLA*_s000014*']
patterns = ['*cryomill_P3HB_powder*_scan*',
 '*avg_P3HB_*_50RPM*scan*',
 '*avg_P3HB_*_100RPM*scan*',
 '*avg_P3HB_*_200RPM*scan*',
 '*avg_P3HB_*_400RPM*scan*',
 '*avg_P3HB_*_600RPM*scan*'
]
#avg_P3HB_03_100RPM_rep

read_and_plot_chi_intensity(folder, patterns)


patterns = ['*cryomill_PLLA_powder*_scan*',
 '*PLLA_08_50rpm_80C_tension*',
    '*PLLA_07_50rpm_80C_free*',]
read_and_plot_chi_intensity(folder, patterns)

patterns = [#'*cryomill*PLLA*',
    '*PLLA*50rpm*as_spun*',
    '*PLLA*100rpm*as_spun*',
    '*PLLA*200rpm*as_spun*',
    '*PLLA*400rpm*as_spun*',
    '*PLLA*600rpm*as_spun*',
    
]


read_and_plot_chi_intensity(folder, patterns)





patterns = ['*cryomill_P3HB_powder*_scan*',
 '*P3HB_07_50rpm_80C_*scan*',
    '*P3HB_08_50rpm_3xcold_draw_*scan*',
    '*P3HB_09_50rpm_3xcolddraw_80C_*scan*',

]
read_and_plot_chi_intensity(folder, patterns)


patterns = ['*cryomill_PET_powder*',
 '*avg_PET_*_50rpm*',
 '*avg_PET_*_100rpm*',
 '*avg_PET_*_200rpm*',
 '*avg_PET_*_400rpm*',
 '*avg_PET_*_600rpm*'
]
read_and_plot_chi_intensity(folder, patterns)


patterns = ['*cryomill_PET_powder*',
 '*PET_10_50rpm_3x_cold_draw*',
 '*PET_09_50rpm_3x_cold_draw*',
 '*PET_08_50rpm_120C*',
 #'*PET_07_400rpm_3xcolddraw*',
]
read_and_plot_chi_intensity(folder, patterns)


## Radial 1D Plot

In [ ]:
import os
import fnmatch
import matplotlib.pyplot as plt

def read_and_plot_chi_intensity(folder_path, patterns, normalize=False):
    """
    Searches for files matching patterns in the folder, reads Chi vs I_radial data, and plots.
    
    Parameters:
    - folder_path (str): Path to the folder containing data files
    - patterns (list): List of wildcard patterns to match filenames
    - normalize (bool): If True, normalize the intensity values
    """
    plt.figure(figsize=(6.5, 6), dpi=300)
    plt.clf()
    plt.ion()  # Enable interactive mode

    for filename in os.listdir(folder_path):
        if (
            filename.endswith('.txt')
            and 'radial' in filename
            and not filename.startswith('._')
            and any(fnmatch.fnmatch(filename, pattern) for pattern in patterns)
        ):
            file_path = os.path.join(folder_path, filename)

            chi = []
            intensity = []

            try:
                with open(file_path, 'r') as file:
                    for line in file:
                        # Skip header or empty lines
                        if line.strip() == '' or line.startswith('#'):
                            continue

                        parts = line.strip().split()
                        if len(parts) >= 2:
                            chi.append(float(parts[0]))
                            intensity.append(float(parts[1]))

                if not chi:
                    print(f"No data found in {filename}. Skipping.")
                    continue
                # mask intesity in given chi mask in -82 to -75 but pass out of of this range
                chi = np.array(chi)
                intensity = np.array(intensity)
                mask = (chi <= -80) | (chi >= -74)
                chi = chi[mask]
                intensity = intensity[mask]
                # Optional normalization
                if normalize:
                    max_intensity = max(intensity)
                    #intensity = [i / max_intensity for i in intensity]

                # Normalize intensity by sum of total intensity
                intensity = np.array(intensity)
                intensity = 10000 * intensity / np.sum(intensity)

                # Use part of the filename as label
                #label = filename.split('_')[2]  # Adjust index if needed
                label = filename.split('.')[0]  # Adjust if needed
                print(f"Filename: {label}")
                label = label.replace('thresh1_', '')
                label = label.split('_01_Scan')[0]
                if 'f_' in label:
                    label = label.split('f_')[1]
                
                if 'P3HB' in label:
                    label = label.replace('P3HB', 'PHA-3B')

                if 'rep_' in label:
                    label = label.replace('rep_', '')
                label = label.replace('_01', '')
                
                plt.plot(chi, intensity,'-o',label=label, markersize=2)

            except Exception as e:
                print(f"Failed to process {file_path}: {e}")

    plt.xlabel('Chi (degrees)')
    plt.ylabel('Intensity (a.u.)')
    plt.xlim(-120, -60)
    #plt.ylim(2, 10)
    set_plot_style(plt.gca(), 20, 'Chi [degrees]', 'Intensity [a.u.]')
    plt.legend()
    plt.tight_layout()
    plt.show()



folder = '/Volumes/SSD1/RawData1/Redesigned_Plastics/Dec2024/D_txt/'
patterns = [#'*cryomill*PLLA*_s00005*',
    '*PLLA*50rpm*s00010*',
    '*PLLA*100rpm*s00010*',
    '*PLLA*200rpm*s00009*',
    '*PLLA*400rpm*s00014*',
    '*PLLA*600rpm*s00010*',
    
]
read_and_plot_chi_intensity(folder, patterns, normalize=True)    

patterns = [#'*cryomill_P3HB_powder*_s00010*',
 '*f_P3HB_*_50RPM*_s00010*',
 '*f_P3HB_*_100RPM*_s00012*',
 '*f_P3HB_*_200RPM*_Scan00005_s00010*',
 '*f_P3HB_*_400RPM*_s00005*',
 '*f_P3HB_*_600RPM*_s00007*'
]

read_and_plot_chi_intensity(folder, patterns, normalize=True) 

patterns = [#'*cryomill_PET_powder*_s00010*',
 '*PET_*_50rpm*_s00013*',
 '*PET_*_100rpm*_s00004*',
 '*PET_*_200rpm*_s00001*',
 '*PET_*_400rpm*_s00007*',
 '*PET_*_600rpm*_s00005*'
]

read_and_plot_chi_intensity(folder, patterns, normalize=True)     

